### Basic example: create a table, modify, import in pandas

we have to restart the kernel each time we want to start again the code

In [1]:
import sqlite3
conn=sqlite3.connect('example.db')
cur=conn.cursor()
cur.executescript('''
DROP TABLE IF EXISTS Instructor;
create table IF NOT EXISTS INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2));
insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA'),(2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US');
''')

now we pull out the table we have just created

In [2]:
#notice that the following code is wrong, because
#the second fetchall() is an empty list

#cur.execute('SELECT * FROM Instructor')
#print(cur.fetchall()) # it's a list!
#for x in cur.fetchall():
 #   print(x)

cur.execute('SELECT * FROM Instructor')
list_of_all_table=cur.fetchall() # it's a list!
for x in list_of_all_table:
    print(x)

print('\n')

cur.execute('SELECT * FROM Instructor')
list_of_some_table=cur.fetchmany(2) # it's a list!
for x in list_of_some_table:
    print(x)

(1, 'Rav', 'Ahuja', 'TORONTO', 'CA')
(2, 'Raul', 'Chong', 'Markham', 'CA')
(3, 'Hima', 'Vasudevan', 'Chicago', 'US')


(1, 'Rav', 'Ahuja', 'TORONTO', 'CA')
(2, 'Raul', 'Chong', 'Markham', 'CA')


Put into a pandas. One way is to construct aa dictionary out of list_of_all_table=cur.fetchall(). Alternatively, we can use a built-in function of pandas:

In [3]:
import pandas as pd

df = pd.read_sql_query('SELECT * FROM Instructor;', conn)
df

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,TORONTO,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [4]:
conn.close()

### magic SQL

In [15]:
import sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

%load_ext sql
conn = sqlite3.connect("example.db")
cur = conn.cursor()
%sql sqlite:///example.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [5]:
%%sql

DROP TABLE IF EXISTS Instructor;
create table IF NOT EXISTS INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2));
insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA'),(2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US');

 * sqlite:///example.db
Done.
Done.
3 rows affected.


[]

In [11]:
%%sql


SELECT * FROM Instructor;


 * sqlite:///example.db
Done.


ID,FNAME,LNAME,CITY,CCODE
1,Rav,Ahuja,TORONTO,CA
2,Raul,Chong,Markham,CA
3,Hima,Vasudevan,Chicago,US


In [12]:
tor='TORONTO'
%sql SELECT * FROM Instructor WHERE CITY=:tor

 * sqlite:///example.db
Done.


ID,FNAME,LNAME,CITY,CCODE
1,Rav,Ahuja,TORONTO,CA


In [13]:
x=%sql SELECT * FROM Instructor WHERE CITY=:tor

x

 * sqlite:///example.db
Done.


ID,FNAME,LNAME,CITY,CCODE
1,Rav,Ahuja,TORONTO,CA


In [17]:
conn.close()

## real examples

### example 1

In [43]:
import sqlite3
import pandas as pd
import prettytable
prettytable.DEFAULT = 'DEFAULT'
%load_ext sql

url='https://data.cityofchicago.org/resource/jcxq-k9xf.csv'
df = pd.read_csv(url) #it's a pandas table

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [44]:
#now we put this table into a database. We create the db and load the df into a table called chicago_socioeconomic_data

conn = sqlite3.connect("socioeconomic.db")
cur = conn.cursor()
%sql sqlite:///socioeconomic.db
df.to_sql('chicago_socioeconomic_data',conn,if_exists='replace',index=False, method='multi')

#control if everything is ok

print(df.head(3).iloc[:,0:3],'\n\n')
cur.execute('SELECT ca,community_area_name,percent_of_housing_crowded FROM chicago_socioeconomic_data LIMIT 3;')
print(cur.fetchall(),'\n\n')
print(pd.read_sql_query('SELECT ca,community_area_name,percent_of_housing_crowded FROM chicago_socioeconomic_data LIMIT 3;', conn))

    ca community_area_name  percent_of_housing_crowded
0  1.0         Rogers Park                         7.7
1  2.0          West Ridge                         7.8
2  3.0              Uptown                         3.8 


[(1.0, 'Rogers Park', 7.7), (2.0, 'West Ridge', 7.8), (3.0, 'Uptown', 3.8)] 


    ca community_area_name  percent_of_housing_crowded
0  1.0         Rogers Park                         7.7
1  2.0          West Ridge                         7.8
2  3.0              Uptown                         3.8


Questions: 
1. How many rows are in the dataset?
2. How many community areas in Chicago have a hardship index greater than 50.0?
3. What is the maximum value of hardship index in this dataset?¶
4. Which community area which has the highest hardship index?
5. Which Chicago community areas have per-capita incomes greater than $60,000?

In [54]:
#Q1

print(df.shape[0],'\n\n\n',pd.read_sql_query('SELECT COUNT(*) FROM chicago_socioeconomic_data;', conn),'\n\n\n')

%sql SELECT COUNT(*) FROM chicago_socioeconomic_data;

78 


    COUNT(*)
0        78 



   sqlite:///example.db
 * sqlite:///socioeconomic.db
Done.


COUNT(*)
78


In [61]:
#Q2

%sql SELECT COUNT(community_area_name) FROM chicago_socioeconomic_data WHERE hardship_index>50.0;

   sqlite:///example.db
 * sqlite:///socioeconomic.db
Done.


COUNT(community_area_name)
38


In [62]:
#Q3

print(df['hardship_index'].max(),'\n\n\n')

%sql SELECT MAX(hardship_index) FROM chicago_socioeconomic_data;

98.0 



   sqlite:///example.db
 * sqlite:///socioeconomic.db
Done.


MAX(hardship_index)
98.0


In [77]:
#Q4

print(df['community_area_name'][df['hardship_index'].idxmax()])
%sql SELECT community_area_name FROM chicago_socioeconomic_data WHERE hardship_index=98.0;
%sql SELECT community_area_name FROM chicago_socioeconomic_data WHERE hardship_index=(SELECT MAX(hardship_index) FROM chicago_socioeconomic_data);

Riverdale
   sqlite:///example.db
 * sqlite:///socioeconomic.db
Done.
   sqlite:///example.db
 * sqlite:///socioeconomic.db
Done.


community_area_name
Riverdale


In [79]:
#Q5

%sql SELECT community_area_name FROM chicago_socioeconomic_data WHERE per_capita_income_>=60000;

   sqlite:///example.db
 * sqlite:///socioeconomic.db
Done.


community_area_name
Lake View
Lincoln Park
Near North Side
Loop


### example 2

In [1]:
import sqlite3
import pandas as pd
import prettytable
prettytable.DEFAULT = 'DEFAULT'
%load_ext sql

url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01'
df = pd.read_csv(url) #it's a pandas table

In [19]:
#now we put this table into a database. We create the db and load the df into a table called chicago_socioeconomic_data

conn = sqlite3.connect("RealWorldData.db")
cur = conn.cursor()
%sql sqlite:///RealWorldData.db
df.to_sql('CHICAGO_PUBLIC_SCHOOLS_DATA',conn,if_exists='replace',index=False)

#control if everything is ok ,Elementary, Middle, or High School

print(df.head(3).iloc[:,0:3],'\n\n')
cur.execute('SELECT School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School" \
            FROM CHICAGO_PUBLIC_SCHOOLS_DATA LIMIT 3;')
print(cur.fetchall(),'\n\n')
print(pd.read_sql_query('SELECT School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School" \
                        FROM CHICAGO_PUBLIC_SCHOOLS_DATA LIMIT 3;', conn))



   School_ID                                     NAME_OF_SCHOOL  \
0     610038                  Abraham Lincoln Elementary School   
1     610281  Adam Clayton Powell Paideia Community Academy ...   
2     610185                Adlai E Stevenson Elementary School   

  Elementary, Middle, or High School  
0                                 ES  
1                                 ES  
2                                 ES   


[(610038, 'Abraham Lincoln Elementary School', 'ES'), (610281, 'Adam Clayton Powell Paideia Community Academy Elementary School', 'ES'), (610185, 'Adlai E Stevenson Elementary School', 'ES')] 


   School_ID                                     NAME_OF_SCHOOL  \
0     610038                  Abraham Lincoln Elementary School   
1     610281  Adam Clayton Powell Paideia Community Academy ...   
2     610185                Adlai E Stevenson Elementary School   

  Elementary, Middle, or High School  
0                                 ES  
1                             

In [22]:
# type in your query to retrieve list of all tables in the database

%sql SELECT name FROM sqlite_master WHERE type='table'


 * sqlite:///RealWorldData.db
Done.


name
CHICAGO_PUBLIC_SCHOOLS_DATA


In [49]:
# type in your query to retrieve the number of columns in the SCHOOLS table
print(df.shape[1],'\n\n\n')
%sql PRAGMA table_info([CHICAGO_PUBLIC_SCHOOLS_DATA]) 

78 



 * sqlite:///RealWorldData.db
Done.


cid,name,type,notnull,dflt_value,pk
0,School_ID,INTEGER,0,None,0
1,NAME_OF_SCHOOL,TEXT,0,None,0
2,"Elementary, Middle, or High School",TEXT,0,None,0
3,Street_Address,TEXT,0,None,0
4,City,TEXT,0,None,0
5,State,TEXT,0,None,0
6,ZIP_Code,INTEGER,0,None,0
7,Phone_Number,TEXT,0,None,0
8,Link,TEXT,0,None,0
9,Network_Manager,TEXT,0,None,0


Questions:

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and parenthesis (round brackets) have been replaced by the underscore character "\_"?


Questions:

1. How many Elementary Schools are in the dataset?
2. What is the highest Safety Score?
3. Which schools have highest Safety Score?
4. What are the top 10 schools with the highest "Average Student Attendance"?
5. Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance
6. Now remove the '%' sign from the above result set for Average Student Attendance column
7. Which Schools have Average Student Attendance lower than 70%?
8. Get the total College Enrollment for each Community Area
9. Get the 5 Community Areas with the least total College Enrollment sorted in ascending order
10. List 5 schools with lowest safety score.
11. Get the hardship index for the community area of the school which has College Enrollment of 4368
12. Get the hardship index for the community area which has the highest value for College Enrollment

In [44]:
#Q1

%sql SELECT COUNT(*) FROM CHICAGO_PUBLIC_SCHOOLS_DATA WHERE "Elementary, Middle, or High School"='ES'

 * sqlite:///RealWorldData.db
Done.


COUNT(*)
462


In [50]:
#Q2

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from CHICAGO_PUBLIC_SCHOOLS_DATA

 * sqlite:///RealWorldData.db
Done.


MAX_SAFETY_SCORE
99.0


In [47]:
#Q3

%sql select NAME_OF_SCHOOL from CHICAGO_PUBLIC_SCHOOLS_DATA where Safety_Score=99.0 LIMIT 3

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL
Abraham Lincoln Elementary School
Alexander Graham Bell Elementary School
Annie Keller Elementary Gifted Magnet School


In [58]:
#Q4

%sql select NAME_OF_SCHOOL, Average_Student_Attendance from CHICAGO_PUBLIC_SCHOOLS_DATA ORDER BY Average_Student_Attendance DESC LIMIT 3

 * sqlite:///RealWorldData.db
Done.


NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE
John Charles Haines Elementary School,98.40%
James Ward Elementary School,97.80%
Edgar Allan Poe Elementary Classical School,97.60%


### example 3 (from the exam)

In [1]:
import sqlite3
import pandas as pd
import prettytable
prettytable.DEFAULT = 'DEFAULT'
%load_ext sql

url1='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01'
url2='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01'
url3='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01'
df1 = pd.read_csv(url1) #it's a pandas table
df2 = pd.read_csv(url2)
df3 = pd.read_csv(url3)


In [6]:
#now we put this table into a database. We create the db and load the df into a table called chicago_socioeconomic_data

conn = sqlite3.connect("FinalDB.db")
cur = conn.cursor()
%sql sqlite:///FinalDB.db
df1.to_sql('CENSUS_DATA',conn,if_exists='replace',index=False)
df2.to_sql('CHICAGO_PUBLIC_SCHOOLS',conn,if_exists='replace',index=False)
df3.to_sql('CHICAGO_CRIME_DATA',conn,if_exists='replace',index=False)


533

In [9]:
#control if everything is ok ,Elementary, Middle, or High School

print(df2.head(3).iloc[:,0:3],'\n\n')
cur.execute('SELECT School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School" \
            FROM CHICAGO_PUBLIC_SCHOOLS LIMIT 3;')
print(cur.fetchall(),'\n\n')
print(pd.read_sql_query('SELECT School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School" \
                        FROM CHICAGO_PUBLIC_SCHOOLS LIMIT 3;', conn))

   School_ID                                     NAME_OF_SCHOOL  \
0     610038                  Abraham Lincoln Elementary School   
1     610281  Adam Clayton Powell Paideia Community Academy ...   
2     610185                Adlai E Stevenson Elementary School   

  Elementary, Middle, or High School  
0                                 ES  
1                                 ES  
2                                 ES   


[(610038, 'Abraham Lincoln Elementary School', 'ES'), (610281, 'Adam Clayton Powell Paideia Community Academy Elementary School', 'ES'), (610185, 'Adlai E Stevenson Elementary School', 'ES')] 


   School_ID                                     NAME_OF_SCHOOL  \
0     610038                  Abraham Lincoln Elementary School   
1     610281  Adam Clayton Powell Paideia Community Academy ...   
2     610185                Adlai E Stevenson Elementary School   

  Elementary, Middle, or High School  
0                                 ES  
1                             

In [16]:
#Q1

print(df3.shape,'\n\n\n')

%sql SELECT COUNT(ID) FROM CHICAGO_CRIME_DATA

(533, 21) 



 * sqlite:///FinalDB.db
Done.


COUNT(ID)
533


In [19]:
#Q2

%sql SELECT COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME from CENSUS_DATA WHERE PER_CAPITA_INCOME<11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
26.0,West Garfield Park
30.0,South Lawndale
37.0,Fuller Park
54.0,Riverdale


In [82]:
#Q4

%sql SELECT * FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE='Kidnapping'\
        and description like='%child%'


 * sqlite:///FinalDB.db
(sqlite3.OperationalError) near "=": syntax error
[SQL: SELECT * FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE='Kidnapping' and description like='%child%']
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [85]:
#Q4

%sql select * from chicago_crime_data where primary_type='KIDNAPPING'\
            and DESCRIPTION like '%child%'

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


In [30]:
#Q5

%sql SELECT DISTINCT(PRIMARY_TYPE) FROM CHICAGO_CRIME_DATA where location_description LIKE "SCHOOL%"

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


In [37]:
#Q6

%sql SELECT AVG(Safety_score) from CHICAGO_PUBLIC_SCHOOLS where "Elementary, Middle, or High School"='ES'

 * sqlite:///FinalDB.db
Done.


AVG(Safety_score)
49.52038369304557


In [36]:
%sql SELECT AVG(Safety_score) from CHICAGO_PUBLIC_SCHOOLS where "Elementary, Middle, or High School"='MS'

 * sqlite:///FinalDB.db
Done.


AVG(Safety_score)
48.0


In [38]:
%sql SELECT AVG(Safety_score) from CHICAGO_PUBLIC_SCHOOLS where "Elementary, Middle, or High School"='HS'

 * sqlite:///FinalDB.db
Done.


AVG(Safety_score)
49.62352941176471


In [39]:
#Q7

%sql SELECT community_area_name,percent_households_below_poverty from census_data order by percent_households_below_poverty desc limit 5

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


In [61]:
#Q8

%sql SELECT COMMUNITY_AREA_NUMBER FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0


In [59]:
#Q9

%sql SELECT COMMUNITY_AREA_NAME from census_data where hardship_index=(select max(hardship_index) from census_data)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


In [80]:
#Q10

%sql SELECT COMMUNITY_AREA_NAME,census_data.community_area_number \
            from census_data,CHICAGO_CRIME_DATA where \
            census_data.community_area_number=CHICAGO_CRIME_DATA.community_area_number\
            AND census_data.community_area_number=25 LIMIT 1




 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
Austin,25.0
